## Welcome to CS4TB3 Group 14 Final Project Demo

### Via this application, we aim to convert Regular Expressions into Simplified Natural Language

In [1]:
# Setting up API
import requests
baseURL = "https://ourapi.com"

In [6]:
print("Enter Regex to convert")
inputRegex = input()
while(not inputRegex):
    print("Please enter a regex to convert")
    inputRegex = input()

Enter Regex to convert

Please enter a regex to convert

Please enter a regex to convert
2


In [7]:
resp = requests.get(f"{baseURL}/?regex={inputRegex}")
print(resp.text)

ConnectionError: HTTPSConnectionPool(host='api.com', port=443): Max retries exceeded with url: /?regex=2 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000237104EF100>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))